In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Sales data

In [ ]:
sales_train_df = pd.read_csv('../input/rossmann-store-sales/train.csv');

In [ ]:
sales_train_df.shape

In [ ]:
sales_train_df.head()

In [ ]:
sales_train_df['DayOfWeek'].unique()

In [ ]:
sales_train_df['Open'].unique()

In [ ]:
sales_train_df['Promo'].unique()

In [ ]:
sales_train_df['StateHoliday'].unique()

In [ ]:
sales_train_df['SchoolHoliday'].unique()

In [ ]:
sales_train_df.tail()

In [ ]:
sales_train_df.info()

In [ ]:
sales_train_df.describe()

### Store data

In [ ]:
store_info_df = pd.read_csv('../input/rossmann-store-sales/store.csv')

In [ ]:
store_info_df.shape

In [ ]:
store_info_df.head()

In [ ]:
store_info_df.info()

In [ ]:
store_info_df.describe()

### Data exploration

#### Sales data

In [ ]:
sns.heatmap(sales_train_df.isnull());

In [ ]:
sales_train_df.isnull().sum()

In [ ]:
sales_train_df.hist(bins = 30, figsize = (20, 20), color = 'r');

In [ ]:
# Number of transactions per day in closed/open stores
closed_train_df = sales_train_df[sales_train_df['Open'] == 0]
open_train_df = sales_train_df[sales_train_df['Open'] == 1]
print('Total = ', len(sales_train_df))
print('number of stores/day closed = ', len(closed_train_df))
print('number of stores/day open = ', len(open_train_df))

In [ ]:
# A better estimate: 154 days in total that stores were closed
172817 / len(store_info_df)

In [ ]:
closed_train_df.head()

We are going to apply a filter so that we only have data from stores on open days, as it makes no sense to use data from when stores are closed, since there are no sales on those days, in addition to messing up the data statistics.

In [ ]:
sales_train_df = sales_train_df[sales_train_df['Open'] == 1]

In [ ]:
sales_train_df.shape

In [ ]:
sales_train_df.head()

In [ ]:
sales_train_df.drop(['Open'], axis = 1, inplace = True)

In [ ]:
sales_train_df.head()

Now, in addition to being more readable, the averages and totals are correct.

In [ ]:
sales_train_df.describe()

#### Stores data

In [ ]:
sns.heatmap(store_info_df.isnull(), cbar = False);

In [ ]:
store_info_df[store_info_df['CompetitionDistance'].isnull()]

In [ ]:
store_info_df[store_info_df['CompetitionOpenSinceMonth'].isnull()]

In [ ]:
store_info_df[store_info_df['CompetitionOpenSinceYear'].isnull()]

In [ ]:
# Replace null values with zero
str_cols = ['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval',
            'CompetitionOpenSinceYear', 'CompetitionOpenSinceMonth']
for str in str_cols:
  store_info_df[str].fillna(0, inplace=True)

In [ ]:
sns.heatmap(store_info_df.isnull(), cbar = False)

In [ ]:
store_info_df.isnull().sum()

In [ ]:
store_info_df['CompetitionDistance'].fillna(store_info_df['CompetitionDistance'].mean(), inplace = True)

In [ ]:
store_info_df.isnull().sum()

In [ ]:
store_info_df.hist(bins = 30, figsize = (20, 20), color = 'r');

It is noticed that the distance from the stores is concentrated around 3 kilometers

#### Merged data (sales + stores)

In [ ]:
sales_train_df.head()

In [ ]:
store_info_df.head()

In [ ]:
sales_train_all_df = pd.merge(sales_train_df, store_info_df, how = 'inner', on = 'Store')

In [ ]:
sales_train_all_df.shape

In [ ]:
sales_train_all_df.tail()

In [ ]:
correlations = sales_train_all_df.corr()
f, ax = plt.subplots(figsize = (20, 20))
sns.heatmap(correlations, annot = True);

In [ ]:
# correlation of sales with other attributes
# A negative value indicates a drop in sales and vice versa. 
# For example, the more promotions, the more sales, and the more competitors, the less sales.
correlations = sales_train_all_df.corr()['Sales'].sort_values()
correlations

In [ ]:
# Separating date attributes into year, month and day.
sales_train_all_df['Year'] = pd.DatetimeIndex(sales_train_all_df['Date']).year
sales_train_all_df['Month'] = pd.DatetimeIndex(sales_train_all_df['Date']).month
sales_train_all_df['Day'] = pd.DatetimeIndex(sales_train_all_df['Date']).day

In [ ]:
sales_train_all_df.head()

In [ ]:
axis = sales_train_all_df.groupby('Month')[['Sales']].mean()
fig = px.line(axis, x = axis.index, y = 'Sales', title = 'Average sales per month', markers = True)
fig.show()

In [ ]:
axis = sales_train_all_df.groupby('Month')[['Customers']].mean()
fig = px.line(axis, x = axis.index, y = 'Customers', title = 'Average customers per month', markers = True)
fig.show()

In [ ]:
axis = sales_train_all_df.groupby('Day')[['Sales']].mean()
fig = px.line(axis, x = axis.index, y = 'Sales', title = 'Average sales per day', markers = True)
fig.show()

In [ ]:
axis = sales_train_all_df.groupby('Day')[['Customers']].mean()
fig = px.line(axis, x = axis.index, y = 'Customers', title = 'Average customers per day', markers = True)
fig.show()

In [ ]:
axis = sales_train_all_df.groupby('DayOfWeek')[['Sales']].mean()
fig = px.line(axis, x = axis.index, y = 'Sales', title = 'Average sales per day of the week', markers = True)
fig.show()

In [ ]:
axis = sales_train_all_df.groupby('DayOfWeek')[['Customers']].mean()
fig = px.line(axis, x = axis.index, y = 'Customers', title = 'Average customers per day of the week', markers = True)
fig.show()

In [ ]:
fig, ax = plt.subplots(figsize = (20,10))
sales_train_all_df.groupby(['Date', 'StoreType']).mean()['Sales'].unstack().plot(ax = ax)

Type b store has the highest average sales, while type a store has the lowest average sales.

In [ ]:
sns.barplot(x = 'Promo', y = 'Sales', data = sales_train_all_df).set_title('Sales with promotion')

In [ ]:
sns.barplot(x = 'Promo', y = 'Customers', data = sales_train_all_df).set_title('Customers with promotion')

### Sales forecasts

In [ ]:
!pip install fbprophet

In [ ]:
from fbprophet import Prophet

In [ ]:
def sales_prediction(store_id, sales_df, periods):
  sales_df = sales_df[sales_df['Store'] == store_id]
  sales_df = sales_df[['Date', 'Sales']].rename(columns = {'Date': 'ds', 'Sales': 'y'})
  sales_df = sales_df.sort_values(by = 'ds')

  model = Prophet()
  model.fit(sales_df)
  future = model.make_future_dataframe(periods = periods)
  forecast = model.predict(future)
  figure1 = model.plot(forecast, xlabel = 'Date', ylabel = 'Sales')
  figure2 = model.plot_components(forecast)

  return sales_df, forecast

In [ ]:
df_origin, df_prediction = sales_prediction(10, sales_train_all_df, 60)

In [ ]:
df_prediction.tail(60).to_csv('submission.csv')

### Forecasts with holidays

In [ ]:
def sales_prediction(store_id, sales_df, holidays, periods):
  sales_df = sales_df[sales_df['Store'] == store_id]
  sales_df = sales_df[['Date', 'Sales']].rename(columns = {'Date': 'ds', 'Sales': 'y'})
  sales_df = sales_df.sort_values(by = 'ds')

  model = Prophet(holidays=holidays)
  model.fit(sales_df)
  future = model.make_future_dataframe(periods = periods)
  forecast = model.predict(future)
  figure1 = model.plot(forecast, xlabel = 'Date', ylabel = 'Sales')
  figure2 = model.plot_components(forecast)

  return sales_df, forecast

In [ ]:
sales_train_all_df.head()

In [ ]:
school_holidays = sales_train_all_df[sales_train_all_df['SchoolHoliday'] == 1].loc[:, 'Date'].values
school_holidays.shape

In [ ]:
state_holidays = sales_train_all_df[(sales_train_all_df['StateHoliday'] == 'a') | 
                                    (sales_train_all_df['StateHoliday'] == 'b') |
                                    (sales_train_all_df['StateHoliday'] == 'c')].loc[:,'Date'].values

In [ ]:
state_holidays = pd.DataFrame({'ds': pd.to_datetime(state_holidays),
                               'holiday': 'state_holiday'})
state_holidays

In [ ]:
school_holidays = pd.DataFrame({'ds': pd.to_datetime(school_holidays),
                               'holiday': 'school_holiday'})
school_holidays

In [ ]:
school_state_holidays = pd.concat((state_holidays, school_holidays))
school_state_holidays

In [ ]:
df_original, df_prediction = sales_prediction(10, sales_train_all_df, school_state_holidays, 10)

In [ ]:
df_prediction.tail(10).to_csv('submission_2.csv')